In [ ]:
import numpy as np
from bmcs_shell.api import MATS2DElastic, FETriangularMesh, TriXDomainFE
from ibvpy.bcond import BCDof
from ibvpy.sim.tstep_bc import TStepBC

# Uniaxial tension test

In [ ]:
wb_mesh = FETriangularMesh(
    X_Id = [[0,0,0], [1000,0,0], [1000,1000,0],[0,1000,0]],
    I_Fi = [[0,1,2], [0,2,3]],
)
fixed_dofs = [0,1,2,5,8,9,11]
loaded_dofs = [3,6]

In [ ]:
xdomain = TriXDomainFE(
    integ_factor = 10,
    mesh=wb_mesh
)

In [ ]:
tmodel = MATS2DElastic(E_=28000, nu=0.2)

In [ ]:
u_cntl = 1000
bc_fixed = [BCDof(var= 'u', dof=dof, value=0 )
           for dof in fixed_dofs]
bc_loaded = [BCDof(var= 'f', dof=dof, value=u_cntl )
           for dof in loaded_dofs]

In [ ]:
ts = TStepBC(
    domains=[(xdomain, tmodel)],
    bc=bc_fixed + bc_loaded,
)

In [ ]:
s = ts.sim
s.tloop.k_max = 10
s.tline.step = 1
s.tloop.verbose = True
s.run()

In [ ]:
U1 = ts.hist.U_t[-1]
print(U1)
eps1 = ts.fe_domain[0].xmodel.map_U_to_field(U1)
sig1, _ = ts.fe_domain[0].tmodel.get_corr_pred(eps1,1)
sig_x, sig_y, tau = sig1.T
sig_ab = np.einsum('ab...->...ab', np.array([[sig_x, tau], [tau, sig_y]], dtype=np.float_))
sig_val, sig_dir = np.linalg.eig(sig_ab)
[np.min(sig_val), np.max(sig_val)]


In [ ]:
sig1

In [ ]:
sig_val

In [ ]:
eps1

In [ ]:
3/28000